# Integrantes:
####Juan Felipe Baquero Vargas
####Lina Fernanda Rosales Castro




Se realizan los imports necesarios para la actividad

In [2]:
%matplotlib inline
import sklearn as skl
import numpy as n
import pylab as pl
import pandas as pd

##Punto 1

###A - Expresión utilizando kernerls


Dado un vecto $w$ en $X$, calcular la norma de la proyección de la imagen en un punto $x$ donde  la proyección $\phi(x)$ en el ve 
Recalling the definition of $k$, and that the norm can be defined from the dot product as $\Vert x\Vert = \langle x, x\rangle^\frac{1}{2}$, we obtain that
$$P_{\phi(w)}\left(\phi(x)\right)=k(x,w) k(w,w)^{-\frac{1}{2}}$$